<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing Langchain using Claude via AWS Bedrock Tutorial</h1>

LangChain provides high-level APIs that enable users to build powerful applications in a few lines of code. However, it can be challenging to understand what is going on under the hood and to pinpoint the cause of issues. Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your query engine and surfacing problematic "spans" of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build a simple question and answer app using LangChain that uses retrieval-augmented generation to answer questions over the Arize documentation,
- Record trace data in OpenInference format,
- Inspect the traces and spans of your application to identify sources of latency and cost,
- Export your trace data as a pandas dataframe and run an LLM-assisted evaluation to measure the precision@k of your retrieval step.

ℹ️ This notebook requires valid AWS Credential.

## 1. Install Dependencies and Import Libraries

Install Phoenix, LangChain, and Boto3.

In [ ]:
!pip install "langchain>=0.1.0" "arize-phoenix[evals]" tiktoken nest-asyncio boto3 -q

Import libraries.

In [ ]:
import json
import os
from urllib.request import urlopen

import boto3
import nest_asyncio
import numpy as np
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.retrievers import KNNRetriever
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings
from phoenix.evals import (
    BedrockModel,
    HallucinationEvaluator,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from phoenix.trace.langchain import LangChainInstrumentor
from tqdm import tqdm

nest_asyncio.apply()  # needed for concurrent evals in notebook environments

bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LangChain application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LangChain application).

In [ ]:
session = px.launch_app()

In [ ]:
px.active_session().view()

## 3. Configure Your AWS Credential

Set your AWS Credential if it is not already set as an environment variable.

In [ ]:
from google.colab import userdata

os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("aws_access_key_id")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("aws_secret_access_key")

### 4. Build Your LangChain Application

This example uses a `RetrievalQA` chain over a pre-built index of the Arize documentation, but you can use whatever LangChain application you like.

Download your pre-built index from cloud storage and instantiate your storage context.

In [ ]:
df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/"
    "unstructured/llm/context-retrieval/langchain/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(df["text_vector"]),
    texts=df["text"].tolist(),
    embeddings=BedrockEmbeddings(client=bedrock_client, model_id="amazon.titan-embed-text-v1"),
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank
chat_model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
model_args = {"temperature": 1, "top_k": 250, "top_p": 0.999}
llm = BedrockChat(
    model_kwargs=model_args, model_id=chat_model_name, streaming=False, client=bedrock_client
)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=knn_retriever,
    metadata={"application_type": "question_answering"},
)

Instrument LangChain

In [ ]:
LangChainInstrumentor().instrument()

## 5. Run Your Query Engine and View Your Traces in Phoenix

Download a sample of queries commonly asked of the Arize documentation.

In [ ]:
url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]

Run a few queries.

In [ ]:
for query in tqdm(queries[:10]):
    chain.invoke(query)

Check the Phoenix UI as your queries run. Your traces should appear in real time.

## 6. Export and Evaluate Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In this case, we will export our `retriever` spans into two separate dataframes:
- `queries_df`, in which the retrieved documents for each query are concatenated into a single column,
- `retrieved_documents_df`, in which each retrieved document is "exploded" into its own row to enable the evaluation of each query-document pair in isolation.

This will enable us to compute multiple kinds of evaluations, including:
- relevance: Are the retrieved documents grounded in the response?
- Q&A correctness: Are your application's responses grounded in the retrieved context?
- hallucinations: Is your application making up false information?

In [ ]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

Next, define your evaluation model and your evaluators.

Evaluators are built on top of language models and prompt the LLM to assess the quality of responses, the relevance of retrieved documents, etc., and provide a quality signal even in the absence of human-labeled data. Pick an evaluator type and instantiate it with the language model you want to use to perform evaluations using our battle-tested evaluation templates.

In [ ]:
eval_model = BedrockModel(model_id=chat_model_name, client=bedrock_client)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

Your evaluations should now appear as annotations on the appropriate spans in Phoenix.

In [ ]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")

## 7. Final Thoughts

LLM Traces and the accompanying OpenInference Tracing specification is designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context such as retrieval from vector stores and the usage of external tools such as search engines or APIs. It lets you understand the inner workings of the individual steps your application takes wile also giving you visibility into how your system is running and performing as a whole.

LLM Evals are designed for simple, fast, and accurate LLM-based evaluations. They let you quickly benchmark the performance of your LLM application and help you identify the problematic spans of execution.

For more details on Phoenix, LLM Tracing, and LLM Evals, checkout our [documentation](https://docs.arize.com/phoenix/).